In [5]:
%%writefile lab8_my_pipeline.py
import argparse
import time
import logging
import json
import apache_beam as beam
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.runners import DataflowRunner, DirectRunner

# ### functions and classes

def parse_json(element):
    return json.loads(element)

def drop_fields(element):
    element.pop('user_agent')
    return element

# ### main

def run():
    # Command line arguments
    parser = argparse.ArgumentParser(description='Load from Json into BigQuery')
    parser.add_argument('--project',required=True, help='Specify Google Cloud project')
    parser.add_argument('--region', required=True, help='Specify Google Cloud region')
    parser.add_argument('--runner', required=True, help='Specify Apache Beam Runner')
    parser.add_argument('--inputPath', required=True, help='Path to events.json')
    parser.add_argument('--outputPath', required=True, help='Path to coldline storage bucket')
    parser.add_argument('--tableName', required=True, help='BigQuery table name')

    opts, pipeline_opts = parser.parse_known_args()

    # Setting up the Beam pipeline options
    options = PipelineOptions(pipeline_opts)
    options.view_as(GoogleCloudOptions).project = opts.project
    options.view_as(GoogleCloudOptions).region = opts.region
    options.view_as(GoogleCloudOptions).job_name = '{0}{1}'.format('my-pipeline-',time.time_ns())
    options.view_as(StandardOptions).runner = opts.runner

    input_path = opts.inputPath
    output_path = opts.outputPath
    table_name = opts.tableName

    # Table schema for BigQuery
    table_schema = {
        "fields": [
            {
                "name": "ip",
                "type": "STRING"
            },
            {
                "name": "user_id",
                "type": "STRING"
            },
            {
                "name": "lat",
                "type": "FLOAT",
                "mode": "NULLABLE"
            },
            {
                "name": "lng",
                "type": "FLOAT",
                "mode": "NULLABLE"
            },
            {
                "name": "timestamp",
                "type": "STRING"
            },
            {
                "name": "http_request",
                "type": "STRING"
            },
            {
                "name": "http_response",
                "type": "INTEGER"
            },
            {
                "name": "num_bytes",
                "type": "INTEGER"
            }
        ]
    }

    # Create the pipeline
    p = beam.Pipeline(options=options)

    '''

    Steps:
    1) Read something
    2) Transform something
    3) Write something

    '''

    # Read in lines to an initial PCollection that can then be branched off of
    lines = p | 'ReadFromGCS' >> beam.io.ReadFromText(input_path)

    # Write to Google Cloud Storage
    lines | 'WriteRawToGCS' >> beam.io.WriteToText(output_path)

    # Read elements from Json, filter out individual elements, and write to BigQuery
    (lines
        | 'ParseJson' >> beam.Map(parse_json)
        | 'DropFields' >> beam.Map(drop_fields)
        | 'FilterFn' >> beam.Filter(lambda row: row['num_bytes'] < 120)
        | 'WriteToBQ' >> beam.io.WriteToBigQuery(
            table_name,
            schema=table_schema,
            create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
            write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE
            )
    )

    logging.getLogger().setLevel(logging.INFO)
    logging.info("Building pipeline ...")

    p.run()

if __name__ == '__main__':
    run()

Overwriting lab8_my_pipeline.py


In [8]:
import os

In [9]:
os.environ["workdir"]="/path"

In [10]:
%%bash
cat $workdir/create_batch_sinks.sh

#!/bin/#!/usr/bin/env bash
echo "Creating pipeline sinks"

PROJECT_ID=$(gcloud config get-value project)

# GCS buckets
#TODO: Add try/catch for the first bucket since qwiklabs
gsutil mb -l US gs://$PROJECT_ID
gsutil mb -l US -c "COLDLINE" gs://$PROJECT_ID-coldline

# BiqQuery Dataset
bq mk --location=US logs

In [11]:
%%bash
cat $workdir/generate_batch_events.sh

#!/bin/#!/usr/bin/env bash
echo "Installing packages"
# Install modules
sh ./install_packages.sh

echo "Generating synthetic users"
# Generate 2 fake web site users
python3 user_generator.py --n=10

echo "Generating synthetic events"
rm *.out 2> /dev/null
# Generate 10 events
python3 batch_event_generator.py --num_e=1000

echo "Copying events to Cloud Storage"
# Set BUCKET to the non-coldline Google Cloud Storage bucket
export BUCKET=gs://$(gcloud config get-value project)/
# Copy events.json into the bucket
gsutil cp events.json ${BUCKET}


In [ ]:
%%bash
bash $workdir/create_batch_sinks.sh

In [ ]:
%%bash
bash $workdir/generate_batch_events.sh

In [ ]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [ ]:
os.environ["PROJECT_ID"]=PROJECT_ID
os.environ["REGION"]='us-central1'
os.environ["BUCKET"]="gs://"+PROJECT_ID
os.environ["COLDLINE_BUCKET"]="gs://"+PROJECT_ID+"-coldline"
os.environ["PIPELINE_FOLDER"]="gs://"+PROJECT_ID
os.environ["INPUT_PATH"]="gs://"+PROJECT_ID+"/events.json"
os.environ["OUTPUT_PATH"]="gs://"+PROJECT_ID+"-coldline/pipeline_output"
os.environ["TABLE_NAME"]=PROJECT_ID+":logs.logs_filtered"
os.environ["PUBSUB_TOPIC"]=f"projects/{PROJECT_ID}/topics/my_topic"
os.environ["WINDOW_DURATION"]=60
os.environ["AGGREGATE_TABLE_NAME"]=f"{PROJECT_ID}:logs.windowed_traffic"
os.environ["RAW_TABLE_NAME"]=f"{PROJECT_ID}:logs.raw"
os.environ["RUNNER"]="DataflowRunner"

In [ ]:
# Set up environment variables
# export PROJECT_ID=$(gcloud config get-value project)
# export REGION='us-central1'
# export BUCKET=gs://${PROJECT_ID}
# export COLDLINE_BUCKET=${BUCKET}-coldline
# export PIPELINE_FOLDER=${BUCKET}
# export RUNNER=DataflowRunner
# export INPUT_PATH=${PIPELINE_FOLDER}/events.json
# export OUTPUT_PATH=${PIPELINE_FOLDER}-coldline/pipeline_output
# export TABLE_NAME=${PROJECT_ID}:logs.logs_filtered
python3 lab8_my_pipeline.py \
--project=${PROJECT_ID} \
--region=${REGION} \
--stagingLocation=${PIPELINE_FOLDER}/staging \
--tempLocation=${PIPELINE_FOLDER}/temp \
--runner=${RUNNER} \
--inputPath=${INPUT_PATH} \
--outputPath=${OUTPUT_PATH} \
--tableName=${TABLE_NAME}

In [12]:
cat $workdir/generate_streaming_events.sh

#!/bin/#!/usr/bin/env bash
echo "Installing packages"
# Install modules
sh ./install_packages.sh

echo "Generating synthetic users"
# Generate 10 fake web site users
python3 user_generator.py --n=10

echo "Generating synthetic events"
use_lag=$1

if [ "$use_lag" = true ] ; then
    echo "Using lag"
    python3 streaming_event_generator.py --project_id=$(gcloud config get-value project) -t=my_topic
else
    echo "Not using lag"
    python3 streaming_event_generator.py --project_id=$(gcloud config get-value project) -t=my_topic -off=1. -on=0. -l=0
fi

In [ ]:
bash $workdir/generate_streaming_events.sh